In [51]:
%pip install pandas numpy scikit-learn matplotlib inline

Note: you may need to restart the kernel to use updated packages.


In [52]:
# CHECKING DIRECTORY
import os
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\Gabriel\Documents\GitHub\eq-forecast\notebooks


In [53]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam

In [54]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [55]:
from src.preprocessing.load_catalog import load_catalog

# Load DataFrame and CSEPCatalog object
df, catalog = load_catalog("../data/eq_catalog.csv")

# Preview
print("Columns:", df.columns.tolist())
print("First rows:\n", df.head())


Columns: ['id', 'latitude', 'longitude', 'depth', 'magnitude', 'Date_Time', 'origin_time']
First rows:
    id  latitude  longitude  depth  magnitude                 Date_Time  \
0   0       4.0      127.0    100        7.0 1909-04-25 14:36:00+00:00   
1   1       4.5      126.5     33        7.5 1910-12-16 06:45:00+00:00   
2   2       6.0      126.0    100        6.8 1911-03-06 09:30:00+00:00   
3   3       9.0      126.0     33        7.7 1911-07-11 20:07:36+00:00   
4   4       4.0      127.0     33        7.5 1912-08-17 11:11:48+00:00   

     origin_time  
0 -1915089840000  
1 -1863278100000  
2 -1856356200000  
3 -1845345144000  
4 -1810558092000  


In [56]:
features_df = df[['origin_time', 'latitude', 'longitude', 'depth', 'magnitude']].copy()

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features_df)


In [57]:
Tx = 10   # Lookback window
Ty = 1    # Predict 1 future step

X, y = [], []
for i in range(Tx, len(scaled_features)):
    X.append(scaled_features[i - Tx:i])
    y.append(scaled_features[i, -1])  # magnitude

X = np.array(X)
y = np.array(y).reshape(-1, Ty, 1)
s = X.transpose(0, 2, 1)  # For spatial attention: (samples, features, time)

In [58]:
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))

x_train, x_val, x_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
y_train, y_val, y_test = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]
s_train, s_val, s_test = s[:train_size], s[train_size:train_size+val_size], s[train_size+val_size:]


In [59]:
from src.models.attention import build_attention_model

inp_var = 5
h_s = 32
dropout = 0.2
con_dim = 4

pred_model, prob_model = build_attention_model(Tx, Ty, inp_var, h_s, dropout, con_dim)
pred_model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

In [60]:
s0_train = np.zeros((x_train.shape[0], h_s))
c0_train = np.zeros((x_train.shape[0], h_s))
yhat0_train = np.zeros((x_train.shape[0], 1))

outputs_train = list(y_train.swapaxes(0, 1))  # List of Ty arrays

history = pred_model.fit(
    [x_train, s_train, s0_train, c0_train, yhat0_train],
    outputs_train,
    batch_size=64,
    epochs=10,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.0542 - val_loss: 0.0110
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0159 - val_loss: 0.0104
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0137 - val_loss: 0.0124
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0107 - val_loss: 0.0190
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0089 - val_loss: 0.0162
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0087 - val_loss: 0.0200
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0089 - val_loss: 0.0160
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0089 - val_loss: 0.0141
Epoch 9/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0087 - val_loss: 0.0142
Epoch 10/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0083 - val_loss: 0.0150


In [61]:
s0_test = np.zeros((x_test.shape[0], h_s))
c0_test = np.zeros((x_test.shape[0], h_s))
yhat0_test = np.zeros((x_test.shape[0], 1))

predictions = pred_model.predict([x_test, s_test, s0_test, c0_test, yhat0_test])
attn_outputs = prob_model.predict([x_test, s_test, s0_test, c0_test, yhat0_test])

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step


In [62]:
Ty = 1

alphas = np.array([attn_outputs[i] for i in range(1, Ty * 3, 3)]).squeeze(-1).transpose(1, 0, 2)
betas = np.array([attn_outputs[i] for i in range(2, Ty * 3, 3)]).squeeze(-1).transpose(1, 0, 2)


In [ ]:
sample_idx = 0
plt.figure(figsize=(8, 4))
plt.stem(range(Tx), alphas[sample_idx, 0])
plt.title(f'Temporal Attention - Sample {sample_idx}')
plt.xlabel('Time Step')
plt.ylabel('Weight')
plt.show()


In [ ]:
feature_names = ['origin_time', 'latitude', 'longitude', 'depth', 'magnitude']
plt.bar(feature_names, betas[sample_idx, 0])
plt.title(f'Spatial Attention - Sample {sample_idx}')
plt.ylabel('Weight')
plt.show()


In [ ]:
pred_arr = np.array(predictions).squeeze().reshape(-1, Ty)
true_arr = y_test.reshape(-1, Ty)

# Inverse transform
padded_pred = np.hstack([np.zeros((pred_arr.shape[0], 4)), pred_arr])
padded_true = np.hstack([np.zeros((true_arr.shape[0], 4)), true_arr])
inv_pred = scaler.inverse_transform(padded_pred)[:, -1]
inv_true = scaler.inverse_transform(padded_true)[:, -1]

# Plot
plt.plot(inv_true[:100], label='Actual')
plt.plot(inv_pred[:100], label='Predicted')
plt.title('Predicted vs Actual Magnitude')
plt.xlabel('Sample Index')
plt.ylabel('Magnitude')
plt.legend()
plt.tight_layout()
plt.show()
